In [1]:
import pandas as pd
import numpy as np
import nltk
import pyspark
import matplotlib.pyplot as plt
import os.path as osp
import os
import shutil
from IPython.display import display, HTML

%matplotlib notebook

In [2]:
!which beeline

/opt/anaconda3/bin/beeline


In [3]:
!whoami

matt


In [4]:
!pwd

/home/matt/etl


In [5]:
os.chdir('/home/matt/etl/bill-text/')

In [6]:
!ls

Ready for Markdown abuse? Here we go!

In [7]:
for house in ['hr', 's']:
    for yr in ['113', '114', '115']:
        for sess in ['1', '2']:
            url = f'https://www.govinfo.gov/bulkdata/BILLS/{yr}/{sess}/{house}/BILLS-{yr}-{sess}-{house}.zip'
            cmd = f'wget -O {house}-{yr}-{sess}.zip {url}'
            display(HTML(f'<h2>{cmd}</h2>'))
            !$cmd

--2018-10-13 03:24:11--  https://www.govinfo.gov/bulkdata/BILLS/113/1/hr/BILLS-113-1-hr.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 38787151 (37M) [application/zip]
Saving to: ‘hr-113-1.zip’

100%[======================================>] 38,787,151  1.89MB/s   in 20s    

2018-10-13 03:24:31 (1.89 MB/s) - ‘hr-113-1.zip’ saved [38787151/38787151]



--2018-10-13 03:24:31--  https://www.govinfo.gov/bulkdata/BILLS/113/2/hr/BILLS-113-2-hr.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 31486595 (30M) [application/zip]
Saving to: ‘hr-113-2.zip’

100%[======================================>] 31,486,595  1.96MB/s   in 15s    

2018-10-13 03:24:47 (1.95 MB/s) - ‘hr-113-2.zip’ saved [31486595/31486595]



--2018-10-13 03:24:47--  https://www.govinfo.gov/bulkdata/BILLS/114/1/hr/BILLS-114-1-hr.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 44807667 (43M) [application/zip]
Saving to: ‘hr-114-1.zip’

100%[======================================>] 44,807,667  1.95MB/s   in 22s    

2018-10-13 03:25:10 (1.94 MB/s) - ‘hr-114-1.zip’ saved [44807667/44807667]



--2018-10-13 03:25:10--  https://www.govinfo.gov/bulkdata/BILLS/114/2/hr/BILLS-114-2-hr.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 28801549 (27M) [application/zip]
Saving to: ‘hr-114-2.zip’

100%[======================================>] 28,801,549  1.89MB/s   in 15s    

2018-10-13 03:25:30 (1.87 MB/s) - ‘hr-114-2.zip’ saved [28801549/28801549]



--2018-10-13 03:25:30--  https://www.govinfo.gov/bulkdata/BILLS/115/1/hr/BILLS-115-1-hr.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 47296838 (45M) [application/zip]
Saving to: ‘hr-115-1.zip’

100%[======================================>] 47,296,838  1.88MB/s   in 24s    

2018-10-13 03:26:00 (1.87 MB/s) - ‘hr-115-1.zip’ saved [47296838/47296838]



...

--2018-10-13 03:26:32--  https://www.govinfo.gov/bulkdata/BILLS/113/2/s/BILLS-113-2-s.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 15037458 (14M) [application/zip]
Saving to: ‘s-113-2.zip’

100%[======================================>] 15,037,458  1.95MB/s   in 7.4s   

2018-10-13 03:26:45 (1.94 MB/s) - ‘s-113-2.zip’ saved [15037458/15037458]



--2018-10-13 03:26:45--  https://www.govinfo.gov/bulkdata/BILLS/114/1/s/BILLS-114-1-s.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 25017956 (24M) [application/zip]
Saving to: ‘s-114-1.zip’

100%[======================================>] 25,017,956  1.88MB/s   in 13s    

2018-10-13 03:26:58 (1.88 MB/s) - ‘s-114-1.zip’ saved [25017956/25017956]



--2018-10-13 03:26:58--  https://www.govinfo.gov/bulkdata/BILLS/114/2/s/BILLS-114-2-s.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 22414691 (21M) [application/zip]
Saving to: ‘s-114-2.zip’

100%[======================================>] 22,414,691  1.94MB/s   in 11s    

2018-10-13 03:27:10 (1.93 MB/s) - ‘s-114-2.zip’ saved [22414691/22414691]



--2018-10-13 03:27:10--  https://www.govinfo.gov/bulkdata/BILLS/115/1/s/BILLS-115-1-s.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 21222690 (20M) [application/zip]
Saving to: ‘s-115-1.zip’

100%[======================================>] 21,222,690  1.96MB/s   in 10s    

2018-10-13 03:27:21 (1.95 MB/s) - ‘s-115-1.zip’ saved [21222690/21222690]



--2018-10-13 03:27:21--  https://www.govinfo.gov/bulkdata/BILLS/115/2/s/BILLS-115-2-s.zip
Resolving www.govinfo.gov (www.govinfo.gov)... 162.140.14.21
Connecting to www.govinfo.gov (www.govinfo.gov)|162.140.14.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Syntax error in Set-Cookie: HttpOnly;Secure at position 9.
Length: 14420646 (14M) [application/zip]
Saving to: ‘s-115-2.zip’

100%[======================================>] 14,420,646  1.94MB/s   in 7.1s   

2018-10-13 03:27:29 (1.93 MB/s) - ‘s-115-2.zip’ saved [14420646/14420646]



In [8]:
!ls

hr-113-1.zip  hr-114-1.zip  hr-115-1.zip  s-113-1.zip  s-114-1.zip  s-115-1.zip
hr-113-2.zip  hr-114-2.zip  hr-115-2.zip  s-113-2.zip  s-114-2.zip  s-115-2.zip


OK, now unzip these files...

In [13]:
files = os.listdir()
for f in files:
    !unzip -q {f} -d billtexts/

## Parse them

In [54]:
import re
import lxml.etree as ET

In [79]:
class Bill(object):
    """
    Parses an XML bill file from Congress.
    """
    
    @classmethod
    def __init__(self, xml_file):
        # Escape ampersands
        with open(xml_file, 'r') as f:
            conts = f.read()
        conts = Bill.xml_clean(conts)
        self.congress, self.house, self.billnum, self.extra = self.extract_metainfo(xml_file)
        tree = ET.fromstring(conts)
        self.official_title = ''.join(tree.xpath('//official-title//text()')).replace('&amp;', '&')
        self.text = ' '.join(tree.xpath('//legis-body//text()')).replace('&amp;', '&')
    
    
    @classmethod
    def extract_metainfo(self, xml_file):
        return re.match(r'.*BILLS-(?P<congress>\d{2,3})(?P<house>s|hr)(?P<billnum>\d+)(?P<extra>.*).xml', xml_file).groups()
    
    @staticmethod
    def xml_clean(data):
        """
        Replace ampersands (XML is a broken standard)
        """
        data = re.sub(r'&', r'&amp;', data)
        data = re.sub(r'&amp;amp;', r'&amp;', data)
        return data

In [80]:
import lxml.etree as ET
os.makedirs('billtext_processed', exist_ok=True)

for i, billFile in enumerate(os.listdir('billtexts')):
    bill = Bill(osp.join('billtexts', billFile))
    outName = f'{bill.house}-{bill.congress}-{bill.billnum}-{bill.extra}.txt'
    if osp.exists(osp.join('billtext_processed', outName)):
        raise Exception(f'Bill exists! {outName}')
    with open(osp.join('billtext_processed', outName), 'w') as f:
        f.write(bill.official_title + '\n' + bill.text)

## Clean up

In [81]:
!ls

billtext_processed  hr-113-2.zip  hr-115-1.zip	s-113-2.zip  s-115-1.zip
billtexts	    hr-114-1.zip  hr-115-2.zip	s-114-1.zip  s-115-2.zip
hr-113-1.zip	    hr-114-2.zip  s-113-1.zip	s-114-2.zip


In [82]:
!rm -rf *.zip billtexts

In [83]:
!ls

billtext_processed


In [84]:
!hdfs dfs -ls /

2018-10-13 04:40:02,936 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
drwxrwxrwx   - matt   supergroup          0 2018-10-07 20:10 /staging
drwxrwxrwx   - hadoop supergroup          0 2018-10-07 17:56 /tmp
drwxr-xr-x   - matt   supergroup          0 2018-10-07 23:41 /user


In [85]:
!hdfs dfs -mkdir -p /billtext

2018-10-13 04:40:15,429 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [86]:
!hdfs dfs -put billtext_processed /billtext

2018-10-13 04:40:35,998 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
